In [82]:
from data_processing.Datasets import get_maestro_dataset, collate_fn
from torch.utils.data import DataLoader
import torch
from model.transformer import Transformer
from tqdm import tqdm
from time import sleep
from functools import partial


In [83]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [84]:
# training hyperparams
EPOCHS = 5
LEARNING_RATE = 0.01

# data hyperparams
SEQ_LEN = 700
BATCH_SIZE = 32
EVAL_BATCH_SIZE = 10

# model hyperparams
NUM_TOKENS = 388
DIM_MODEL = 512
NUM_HEADS = 2
NUM_ENCODER_LAYERS = 3
NUM_DECODER_LAYERS = 3
DROPOUT_P = 0.1


In [85]:
# load device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [86]:
train_data, test_data = get_maestro_dataset("data/maestro", representation="pitch")


Skip downloading as the `.muspy.success` file is found.
Skip extracting as the `.muspy.success` file is found.
Skip conversion as the `.muspy.success` file is found.


In [87]:
# Build dataloaders
train_dataloader = DataLoader(
    dataset=train_data,
    batch_size=BATCH_SIZE,
    collate_fn=partial(collate_fn, seq_len=SEQ_LEN, device=device),
    shuffle=False,
)

val_dataloader = DataLoader(
    dataset=test_data,
    batch_size=EVAL_BATCH_SIZE,
    collate_fn=partial(collate_fn, seq_len=SEQ_LEN, device=device),
    shuffle=False,
)


In [88]:
# create model, optim, criterion
model = Transformer(
    num_tokens=NUM_TOKENS,
    dim_model=DIM_MODEL,
    num_heads=NUM_HEADS,
    num_encoder_layers=NUM_ENCODER_LAYERS,
    num_decoder_layers=NUM_DECODER_LAYERS,
    dropout_p=DROPOUT_P,
).to(device)


In [89]:
# load saved model params
model.load_state_dict(
    torch.load("./results/transformer/model_params/transformer_music.pth")
)

# set to test
model.eval()


# get some data, set a sequence length
with torch.no_grad():
    # get first index from val
    batch = next(iter(val_dataloader))

    primer = batch[0]
    primer = primer.long().to(device)
    labels = batch[2].long().to(device)

    primer = primer.permute(1, 0)
    labels = labels.permute(1, 0)
    primer = torch.tensor(primer[:1], dtype=torch.long)
    labels = torch.tensor(labels[:1], dtype=torch.long)
    primer = primer.permute(1, 0)
    labels = labels.permute(1, 0)

    primer = primer[:50]
    labels = primer[:50]

    data = model.generate(primer, device, labels, target_seq_length=300)

# decode the returned info

# use muspy to turn data into mp3
# convert the output into a music object with muspy.from_event_representation()

# call muspy.write_audio(), passing in music file, and format, to format into audio file


/var/folders/3w/lhkpgfc505n81_2vs8svxfpr0000gn/T/ipykernel_75549/2870899523.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  primer = torch.tensor(primer[:1], dtype=torch.long)
/var/folders/3w/lhkpgfc505n81_2vs8svxfpr0000gn/T/ipykernel_75549/2870899523.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels[:1], dtype=torch.long)


torch.Size([50, 1])
Generating Sequence of length 1024


KeyboardInterrupt: 